<a href="https://colab.research.google.com/github/gmsharpe/edumore/blob/master/terraform_aws_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terraform, AWS Athena & Google Colab

## Download and Install Terraform

In [1]:
%%bash
# instructions from https://learn.hashicorp.com/tutorials/terraform/install-cli?in=terraform/aws-get-started
sudo apt-get update && sudo apt-get install -y gnupg software-properties-common curl
curl -fsSL https://apt.releases.hashicorp.com/gpg | sudo apt-key add -
sudo apt-add-repository "deb [arch=amd64] https://apt.releases.hashicorp.com $(lsb_release -cs) main"
sudo apt-get update && sudo apt-get install terraform

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:15 http://ppa

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


## Download and Install the AWS CLI

In [2]:
%%bash

# download the installation file
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"

unzip -q awscliv2.zip

sudo ./aws/install


You can now run: /usr/local/bin/aws --version


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43.4M  100 43.4M    0     0  18.6M      0  0:00:02  0:00:02 --:--:-- 18.6M



## Verify the Installation of the AWS CLI

In [3]:
%%bash
aws -version


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: the following arguments are required: command



In [4]:
# importing os module
import os
  
path = "/content/terraform_tutorial"
  
if not os.path.exists(path):
  os.mkdir(path)

os.chdir(path)
main_tf = open("main.tf","w")
variables_tf = open("variables.tf","w")

variables = """
locals {
  database_s3_bucket_name = "yellow-taxi-ny-983794"
  query_results_s3_bucket_name = "yellow-taxi-ny-983794-workgroup-query-results"
}
"""

main_body = """
terraform {
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 3.27"
    }
  }
  required_version = ">= 0.14.9"
}

provider "aws" {
  profile = "default"
  region  = "us-west-1"
}

resource "aws_s3_bucket" "bucket" {
  bucket = "${local.database_s3_bucket_name}"
}

# Ensure no public access is possible
resource "aws_s3_bucket_public_access_block" "bucket" {
  bucket = aws_s3_bucket.bucket.id

  block_public_acls       = true
  block_public_policy     = true
  ignore_public_acls      = true
  restrict_public_buckets = true
}
"""

main_tf.write(main_body)
main_tf.close()
variables_tf.write(variables)
variables_tf.close()

# uncomment if you want to read and print 'main_tf'
#main_tf = open("main.tf")
#print(main_tf.read())
#main_tf.close()

In [9]:
%%bash
terraform init 

/content/terraform_tutorial

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Using previously-installed hashicorp/aws v3.71.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [10]:
%%bash
cd terraform_tutorial
terraform plan


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_s3_bucket.bucket will be created
  + resource "aws_s3_bucket" "bucket" {
      + acceleration_status         = (known after apply)
      + acl                         = "private"
      + arn                         = (known after apply)
      + bucket                      = "yellow-taxi-ny-983794"
      + bucket_domain_name          = (known after apply)
      + bucket_regional_domain_name = (known after apply)
      + force_destroy               = false
      + hosted_zone_id              = (known after apply)
      + id                          = (known after apply)
      + region                      = (known after apply)
      + request_payer               = (known after apply)
      + tags_all                    = (known after apply)
      + website_domain              = (known

bash: line 1: cd: terraform_tutorial: No such file or directory


In [11]:
%%bash
terraform apply -auto-approve


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_s3_bucket.bucket will be created
  + resource "aws_s3_bucket" "bucket" {
      + acceleration_status         = (known after apply)
      + acl                         = "private"
      + arn                         = (known after apply)
      + bucket                      = "yellow-taxi-ny-983794"
      + bucket_domain_name          = (known after apply)
      + bucket_regional_domain_name = (known after apply)
      + force_destroy               = false
      + hosted_zone_id              = (known after apply)
      + id                          = (known after apply)
      + region                      = (known after apply)
      + request_payer               = (known after apply)
      + tags_all                    = (known after apply)
      + website_domain              = (known

In [12]:
%%bash
wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.csv -o /dev/null

In [13]:
%%bash
pwd
# because we changed the working directory above, actions taken are still from the perspective of the /content/terraform_tutorial directory.  
# TODO - confirm
aws s3 cp /content/terraform_tutorial/yellow_tripdata_2021-01.csv s3://yellow-taxi-ny-983794
aws s3 ls s3://yellow-taxi-ny-983794/

/content/terraform_tutorial
upload: ./yellow_tripdata_2021-01.csv to s3://yellow-taxi-ny-983794/yellow_tripdata_2021-01.csv
2022-01-08 02:07:06  125981363 yellow_tripdata_2021-01.csv


https://www.xerris.com/insights/building-modern-data-warehouses-with-s3-glue-and-athena-part-3/

In [42]:
athena_tf_body = """

resource "aws_athena_database" "yellow_taxi_ny" {
  name   = "yellow_taxi_ny"
  bucket = aws_s3_bucket.bucket.bucket
}

resource "aws_s3_bucket" "query_results" {
  bucket = "${local.query_results_s3_bucket_name}"
  lifecycle_rule {
    enabled = true
    expiration {
      days = 1
    }
  }
}

# Ensure no public access is possible
resource "aws_s3_bucket_public_access_block" "query_results_bucket_block" {
  bucket = aws_s3_bucket.query_results.id

  block_public_acls       = true
  block_public_policy     = true
  ignore_public_acls      = true
  restrict_public_buckets = true
}

resource "aws_athena_workgroup" "workgroup" {
  name = "my_workgroup"
  configuration {
    enforce_workgroup_configuration    = true
    publish_cloudwatch_metrics_enabled = true

    result_configuration {
      output_location = "s3://${aws_s3_bucket.query_results.bucket}/output/"
    }
  }
}

resource "aws_glue_crawler" "crawler" {
  database_name = aws_athena_database.yellow_taxi_ny.name
  name          = "yellow_taxi_ny_crawler"
  role          = aws_iam_role.yellow_taxi_nydb_role.arn

  s3_target {
    path = "s3://${aws_s3_bucket.bucket.bucket}"
  }
}
"""

iam_tf_body = """

resource "aws_iam_role" "yellow_taxi_nydb_role" {
  name               = "yellow_taxi_nydb_role"
  assume_role_policy = data.aws_iam_policy_document.glue-assume-role-policy.json
}

data "aws_iam_policy_document" "glue-assume-role-policy" {
  statement {
    actions = ["sts:AssumeRole"]
    principals {
      type        = "Service"
      identifiers = ["glue.amazonaws.com"]
    }
  }
}

resource "aws_iam_policy" "policy" {
  name        = "yellow_taxi_nydb"
  description = "yellow_taxi_nydb"
  policy      = data.aws_iam_policy_document.policy-document.json

}

data "aws_iam_policy_document" "policy-document" {
  statement {
    actions = [
    "s3:GetBucketLocation", "s3:ListBucket", "s3:ListAllMyBuckets", 
    "s3:GetBucketAcl", "s3:GetObject"]
    resources = [
      "arn:aws:s3:::yellow-taxi-ny-983794",
      "arn:aws:s3:::yellow-taxi-ny-983794/*"
    ]
  }
}

resource "aws_iam_role_policy_attachment" "policy-attachment" {
  role       = aws_iam_role.yellow_taxi_nydb_role.name
  policy_arn = aws_iam_policy.policy.arn
}

resource "aws_iam_role_policy_attachment" "glue-service-role-attachment" {
  role       = aws_iam_role.yellow_taxi_nydb_role.name
  policy_arn = data.aws_iam_policy.AWSGlueServiceRole.arn
}

data "aws_iam_policy" "AWSGlueServiceRole" {
  arn = "arn:aws:iam::aws:policy/service-role/AWSGlueServiceRole"
}

"""

with open("athena.tf","w") as athena_tf:
  athena_tf.writelines(athena_tf_body)

with open("iam.tf","w") as iam_tf:
  iam_tf.writelines(iam_tf_body)

In [43]:
%%bash
#terraform plan
terraform apply -auto-approve

aws_s3_bucket.query_results: Refreshing state... [id=yellow-taxi-ny-983794-workgroup-query-results]
aws_s3_bucket.bucket: Refreshing state... [id=yellow-taxi-ny-983794]
aws_iam_role.yellow_taxi_nydb_role: Refreshing state... [id=yellow_taxi_nydb_role]
aws_iam_policy.policy: Refreshing state... [id=arn:aws:iam::736682772784:policy/yellow_taxi_nydb]
aws_iam_role_policy_attachment.policy-attachment: Refreshing state... [id=yellow_taxi_nydb_role-20220108021055016300000002]
aws_iam_role_policy_attachment.glue-service-role-attachment: Refreshing state... [id=yellow_taxi_nydb_role-20220108021055025600000003]
aws_s3_bucket_public_access_block.bucket: Refreshing state... [id=yellow-taxi-ny-983794]
aws_athena_database.yellow_taxi_ny: Refreshing state... [id=yellow_taxi_ny]
aws_athena_workgroup.workgroup: Refreshing state... [id=my_workgroup]
aws_s3_bucket_public_access_block.query_results_bucket_block: Refreshing state... [id=yellow-taxi-ny-983794-workgroup-query-results]
aws_glue_crawler.crawle

In [44]:
%%bash
aws glue start-crawler --name yellow_taxi_ny_crawler

In [25]:
# install a json string processor (jq)
#!sudo apt install jq >> /dev/null

In [45]:
%%bash
aws glue get-crawler --name yellow_taxi_ny_crawler

{
    "Crawler": {
        "Name": "yellow_taxi_ny_crawler",
        "Role": "yellow_taxi_nydb_role",
        "Targets": {
            "S3Targets": [
                {
                    "Path": "s3://yellow-taxi-ny-983794",
                    "Exclusions": []
                }
            ],
            "JdbcTargets": [],
            "MongoDBTargets": [],
            "DynamoDBTargets": [],
            "CatalogTargets": []
        },
        "DatabaseName": "yellow_taxi_ny",
        "Classifiers": [],
        "RecrawlPolicy": {
            "RecrawlBehavior": "CRAWL_EVERYTHING"
        },
        "SchemaChangePolicy": {
            "UpdateBehavior": "UPDATE_IN_DATABASE",
            "DeleteBehavior": "DEPRECATE_IN_DATABASE"
        },
        "LineageConfiguration": {
            "CrawlerLineageSettings": "DISABLE"
        },
        "State": "RUNNING",
        "CrawlElapsedTime": 32451,
        "CreationTime": "2022-01-08T02:11:01+00:00",
        "LastUpdated": "2022-01-08T02:11:01+0

In [46]:
%%bash
crawler_details=$(aws glue get-crawler --name yellow_taxi_ny_crawler)
#aws glue get-tables --database-name yellow_taxi_ny
#echo $crawler_details
status=$(jq .Crawler.State <<< $crawler_details)
echo $status

"STOPPING"


In [49]:
%%bash
#aws glue list-databases
aws glue get-tables --database-name 'yellow_taxi_ny'

{
    "TableList": [
        {
            "Name": "yellow_taxi_ny_983794",
            "DatabaseName": "yellow_taxi_ny",
            "Owner": "owner",
            "CreateTime": "2022-01-08T02:23:52+00:00",
            "UpdateTime": "2022-01-08T02:23:52+00:00",
            "LastAccessTime": "2022-01-08T02:23:52+00:00",
            "Retention": 0,
            "StorageDescriptor": {
                "Columns": [
                    {
                        "Name": "vendorid",
                        "Type": "bigint"
                    },
                    {
                        "Name": "tpep_pickup_datetime",
                        "Type": "string"
                    },
                    {
                        "Name": "tpep_dropoff_datetime",
                        "Type": "string"
                    },
                    {
                        "Name": "passenger_count",
                        "Type": "bigint"
                    },
                    {
             

In [61]:
%%bash
query_execution_id=$(jq .QueryExecutionId <<< $(aws athena start-query-execution --work-group my_workgroup --query-string "SELECT * FROM yellow_taxi_ny.yellow_taxi_ny_983794 limit 100;"))
echo $query_execution_id

"d2736a78-5fa5-4372-bf17-2b579dfdfaec"


In [71]:
%%bash
query_execution_json=$(aws athena get-query-execution --query-execution-id d2736a78-5fa5-4372-bf17-2b579dfdfaec)
jq .QueryExecution.Status <<< $query_execution_json

{
  "State": "SUCCEEDED",
  "SubmissionDateTime": "2022-01-08T02:45:25.521000+00:00",
  "CompletionDateTime": "2022-01-08T02:45:26.265000+00:00"
}


In [63]:
#!aws athena get-query-results --query-execution-id c561bb4d-d75c-4586-8e9b-e3cd9347bcf3
%%bash
aws s3 mv s3://yellow-taxi-ny-983794-workgroup-query-results/output/d2736a78-5fa5-4372-bf17-2b579dfdfaec.csv /content/results.csv
cat /content/results.csv

move: s3://yellow-taxi-ny-983794-workgroup-query-results/output/d2736a78-5fa5-4372-bf17-2b579dfdfaec.csv to ../results.csv
"vendorid","tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","ratecodeid","store_and_fwd_flag","pulocationid","dolocationid","payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge"
"2","2021-01-10 19:05:32","2021-01-10 19:14:31","2","2.05","1","N","161","246","1","9.0","0.0","0.5","3.69","0.0","0.3","15.99","2.5"
"2","2021-01-10 19:03:03","2021-01-10 19:08:49","1","1.03","1","N","237","262","1","6.0","0.0","0.5","1.0","0.0","0.3","10.3","2.5"
"2","2021-01-10 19:17:12","2021-01-10 19:29:53","1","2.78","1","N","236","166","1","11.5","0.0","0.5","2.96","0.0","0.3","17.76","2.5"
"2","2021-01-10 19:33:13","2021-01-10 19:45:07","1","1.78","1","N","166","75","2","10.0","0.0","0.5","0.0","0.0","0.3","10.8","0.0"
"2","2021-01-10 19:52:14","2021-01-10 19:56:17","1"